In [1]:
import numpy as np
import pandas as pd
import jscatter

In [2]:
n = 500
data = np.random.rand(n, 5)
data[:,3] = np.round(data[:,3] * 7).astype(int)

df = pd.DataFrame(
    data,
    columns=['x', 'y', 'value', 'group', 'bla'],
)
df['group'] = df['group'].astype('int').astype('category').map(lambda c: chr(65 + c), na_action=None)
df.head(5)

,x,y,value,group,bla
0,0.137755,0.271790,0.512896,G,0.597200
1,0.876944,0.935858,0.805132,B,0.428691
2,0.107886,0.252273,0.256287,F,0.957269
3,0.031307,0.902774,0.016379,G,0.151846
4,0.199323,0.784366,0.653936,D,0.996449


## Simple Test

In [3]:
scatter = jscatter.Scatter('x', 'y', df)
scatter.show()

In [4]:
scatter.color(by='group').size(6).height(320).selection(df.query('x < 0.5').index)

In [5]:
scatter.selection()

array([  0,   2,   3,   4,   7,   8,   9,  12,  13,  14,  15,  17,  18,
        19,  22,  23,  24,  25,  27,  29,  30,  33,  35,  36,  37,  38,
        42,  44,  45,  47,  48,  49,  50,  51,  52,  55,  57,  59,  60,
        61,  62,  63,  64,  66,  67,  68,  70,  77,  78,  80,  81,  83,
        84,  86,  87,  89,  90,  93,  94,  95, 100, 101, 105, 106, 107,
       108, 109, 110, 111, 114, 116, 118, 121, 124, 128, 129, 130, 131,
       132, 135, 137, 140, 141, 143, 144, 146, 152, 153, 154, 155, 159,
       160, 163, 164, 165, 167, 168, 169, 171, 172, 173, 174, 175, 178,
       179, 186, 190, 195, 201, 203, 206, 207, 211, 213, 214, 215, 216,
       217, 219, 221, 223, 226, 227, 229, 234, 235, 237, 238, 241, 244,
       245, 246, 249, 250, 251, 252, 253, 255, 258, 259, 260, 261, 263,
       264, 265, 266, 267, 276, 284, 285, 286, 287, 289, 290, 291, 294,
       295, 296, 298, 301, 303, 304, 306, 309, 310, 311, 312, 313, 316,
       318, 322, 323, 324, 325, 327, 329, 330, 333, 341, 343, 34

# Short-Hand Test

In [6]:
jscatter.plot(
    data=df,
    x='x',
    y='y',
    color_by='value',
    color_order='reverse',
    size_by='value',
    size_map=[10,11,12,13,14,15],
    opacity_by='value',
    connect_by='group',
    connection_color_by='value',
    connection_color_order='reverse',
    height=600
)

# Large-Scale Test

In [7]:
def roesslerAttractor(num):
    from math import inf
    
    points = []

    xn = 2.644838333129883
    yn = 4.060488700866699
    zn = 2.8982460498809814
    a = 0.2
    b = 0.2
    c = 5.7
    dt = 0.006

    minX = inf
    maxX = -inf
    minY = inf
    maxY = -inf
    for i in range(num):
        dx = -yn - zn
        dy = xn + a * yn
        dz = b + zn * (xn - c)

        xh = xn + 0.5 * dt * dx
        yh = yn + 0.5 * dt * dy
        zh = zn + 0.5 * dt * dz

        dx = -yh - zh
        dy = xh + a * yh
        dz = b + zh * (xh - c)

        xn1 = xn + dt * dx
        yn1 = yn + dt * dy
        zn1 = zn + dt * dz

        points.append([xn1, yn1])

        minX = min(xn1, minX)
        maxX = max(xn1, maxX)
        minY = min(yn1, minY)
        maxY = max(yn1, maxY)

        xn = xn1;
        yn = yn1;
        zn = zn1;


    dX = maxX - minX
    dY = maxY - minY

    for i in range(num):
        points[i][0] -= minX
        points[i][0] /= dX / 2
        points[i][0] -= 1
        points[i][1] -= minY
        points[i][1] /= dY / 2
        points[i][1] -= 1

    return points

In [8]:
points = np.asarray(roesslerAttractor(1000000))
jscatter.plot(points[:,0], points[:,1])